# User Item Biases With Regularization
* Prediction for user $i$ and item $j$ is $\tilde r_{ij} = u_i + a_j$
* Loss function is $L = \sum_{\Omega}w_{ij}\text{loss}(r_{ij}, \tilde r_{ij}) + \lambda_u \sum_i (u_i - \bar u) ^2 + \lambda_a \sum_j (a_j - \bar a)^2 $
* $\bar u$ is the mean of $u_i$ and $\bar a$ is the mean of $a_j$ 
* $\Omega$ is the set of oberved pairs $(i, j)$
* $r_{ij}$ is the rating for user $i$ and item $j$
* $w_{ij}$ is the weight for the prediction $r_{ij}$ and is modeled as a power-law in the number of items seen by $i$ and users than have seen $j$: $w_{ij} = |j' : (i, j') \in \Omega| ^ {\lambda_{wu}} |i' : (i', j) \in \Omega| ^ {\lambda_{wa}}$
* $\text{loss}$ is mean squared error

In [1]:
const name = "UserItemBiases"
const residual_alphas = []
const implicit = false;

In [2]:
import LineSearches: BackTracking
import NBInclude: @nbinclude
import SparseArrays: sparse
import Statistics: mean
@nbinclude("Alpha.ipynb");

In [3]:
# TODO support residualization
const training = get_split("training", implicit = implicit)
const validation = get_split("validation", implicit = implicit);

## Alternating Least Squares
* Given some hyperparameters $\lambda$, we can solve for $U$ and $A$ via Alternating Least Squares
* This is an iterative algorithm where we fix $A$, then solve for the $U$ that minimizes the loss function
* Then we fix $U$ and solve for the best $A$
* These two steps are repeated until the matrices $U$ and $A$ converge

## ALS for Explicit data
* If we fix $a$, then for each user $i$, $u_i$ is optimized when
* $u_i = \dfrac{\sum_{j \in \Omega_i}(r_{ij} - a_j) w_{ij} + \bar u \lambda_u}{ \sum_{j \in \Omega_i} w_{ij} + \lambda_u}$
* $\Omega$ is the set of (user, item) pairs that we have ratings for
* $\Omega_i$ is subset of $\Omega$ for which the user is the $i$-th user

In [4]:
function make_prediction(users, items, u, a)
    r = Array{eltype(u)}(undef, length(users))
    Threads.@threads for i = 1:length(r)
        @inbounds r[i] = u[users[i]] + a[items[i]]
    end
    r
end;

In [5]:
function get_residuals!(users, items, ratings, weights, a, ρ, Ω)
    @inbounds for row = 1:length(users)
        i = users[row]
        j = items[row]
        r = ratings[row]
        w = weights[row]
        ρ[i] += (r - a[j]) * w
        Ω[i] += w
    end
    ρ, Ω
end

function update_users!(users, items, ratings, weights, u, a, λ_u, ρ, Ω)
    Threads.@threads for t = 1:Threads.nthreads()
        range = thread_range(length(ratings))
        ρ[:, Threads.threadid()] .= 0
        Ω[:, Threads.threadid()] .= 0
        @views get_residuals!(
            users[range],
            items[range],
            ratings[range],
            weights[range],
            a,
            ρ[:, Threads.threadid()],
            Ω[:, Threads.threadid()],
        )
    end

    ρ = sum(ρ, dims = 2)
    Ω = sum(Ω, dims = 2)
    μ = mean(u)
    Threads.@threads for i = 1:length(u)
        @inbounds u[i] = (ρ[i] + μ * λ_u) / (Ω[i] + λ_u)
    end
end;

In [6]:
function train_model(training, stop_criteria, λ)
    @info "training model with parameters $λ"
    λ_u, λ_a, λ_wu, λ_wa = λ
    users, items, ratings = training.user, training.item, training.rating
    weights =
        expdecay(get_counts("training"), log(λ_wu)) .*
        expdecay(get_counts("training"; by_item = true), log(λ_wa))
    u = zeros(eltype(λ_u), num_users())
    a = zeros(eltype(λ_a), num_items())

    ρ_u = zeros(eltype(u), length(u), Threads.nthreads())
    Ω_u = zeros(eltype(u), length(u), Threads.nthreads())
    ρ_a = zeros(eltype(a), length(a), Threads.nthreads())
    Ω_a = zeros(eltype(a), length(a), Threads.nthreads())

    while !stop!(stop_criteria, [u, a])
        update_users!(users, items, ratings, weights, u, a, λ_u, ρ_u, Ω_u)
        update_users!(items, users, ratings, weights, a, u, λ_a, ρ_a, Ω_a)
    end
    u, a
end;

## Training

In [7]:
function validation_mse(λ)
    λ = exp.(λ) # ensure λ is nonnegative
    stop_criteria = convergence_stopper(1e-6, max_iters = 16)
    u, a = train_model(training, stop_criteria, λ)
    r = make_prediction(validation.user, validation.item, u, a)
    residualized_loss(r, residual_alphas, implicit)
end;

In [8]:
# Find the best regularization hyperparameters
res = Optim.optimize(
    validation_mse,
    fill(0.0f0, 4),
    Optim.NewtonTrustRegion(),
    autodiff = :forward,
    Optim.Options(show_trace = true, extended_trace = true),
);
λ = exp.(Optim.minimizer(res));

[ Info: 20220524 23:06:29 training model with parameters ForwardDiff.Dual{ForwardDiff.Tag{typeof(validation_mse), Float32}, Float32, 4}[Dual{ForwardDiff.Tag{typeof(validation_mse), Float32}}(1.0,1.0,0.0,0.0,0.0), Dual{ForwardDiff.Tag{typeof(validation_mse), Float32}}(1.0,0.0,1.0,0.0,0.0), Dual{ForwardDiff.Tag{typeof(validation_mse), Float32}}(1.0,0.0,0.0,1.0,0.0), Dual{ForwardDiff.Tag{typeof(validation_mse), Float32}}(1.0,0.0,0.0,0.0,1.0)]
Progress: 100%|███████████████████████████| Time: 0:00:01 ( 0.34 μs/it)39m
Progress: 100%|███████████████████████████| Time: 0:00:00 (37.44 ns/it)
[ Info: 20220524 23:06:56 regression coefficients: ForwardDiff.Dual{ForwardDiff.Tag{typeof(validation_mse), Float32}, Float32, 4}[Dual{ForwardDiff.Tag{typeof(validation_mse), Float32}}(1.0001966,0.001265263,4.9162406e-7,-0.0022688252,-0.00932882)]
[ Info: 20220524 23:06:59 training model with parameters ForwardDiff.Dual{ForwardDiff.Tag{typeof(validation_mse), Float32}, ForwardDiff.Dual{ForwardDiff.Tag{type

Iter     Function value   Gradient norm 
     0     1.820268e+00     5.538846e-02
 * time: 0.025742053985595703
 * g(x): Float32[-0.005109214, -1.1346601f-6, 0.019632269, 0.05538846]
 * reached_subproblem_solution: true
 * h(x): Float32[0.0042683696 7.7101596f-8 -0.0009896054 -0.045240797; 7.7101575f-8 -8.909593f-7 2.1172305f-7 1.124478f-5; -0.0009896058 2.117212f-7 0.0024552576 0.0019379249; -0.045240793 1.1244779f-5 0.0019380794 0.74176663]
 * x: Float32[0.0, 0.0, 0.0, 0.0]
 * lambda: NaN
 * interior: true
 * hard case: false
 * delta: 1.0


[ Info: 20220524 23:08:16 training model with parameters ForwardDiff.Dual{ForwardDiff.Tag{typeof(validation_mse), Float32}, Float32, 4}[Dual{ForwardDiff.Tag{typeof(validation_mse), Float32}}(1.0515077,1.0515077,0.0,0.0,0.0), Dual{ForwardDiff.Tag{typeof(validation_mse), Float32}}(1.000123,0.0,1.000123,0.0,0.0), Dual{ForwardDiff.Tag{typeof(validation_mse), Float32}}(0.3691843,0.0,0.0,0.3691843,0.0), Dual{ForwardDiff.Tag{typeof(validation_mse), Float32}}(0.9347721,0.0,0.0,0.0,0.9347721)]
[ Info: 20220524 23:08:31 regression coefficients: ForwardDiff.Dual{ForwardDiff.Tag{typeof(validation_mse), Float32}, Float32, 4}[Dual{ForwardDiff.Tag{typeof(validation_mse), Float32}}(0.9958476,-0.0024471707,0.00012416836,0.041969813,0.025591727)]


     1     1.820268e+00     5.538846e-02
 * time: 17.585376024246216
 * g(x): Float32[-0.005109214, -1.1346601f-6, 0.019632269, 0.05538846]
 * reached_subproblem_solution: false
 * h(x): Float32[0.0042683696 7.7101596f-8 -0.0009896054 -0.045240797; 7.7101575f-8 -8.909593f-7 2.1172305f-7 1.124478f-5; -0.0009896058 2.117212f-7 0.0024552576 0.0019379249; -0.045240793 1.1244779f-5 0.0019380794 0.74176663]
 * x: Float32[0.0, 0.0, 0.0, 0.0]
 * lambda: 0.017065741
 * interior: false
 * hard case: false
 * delta: 0.25000045


[ Info: 20220524 23:08:32 training model with parameters ForwardDiff.Dual{ForwardDiff.Tag{typeof(validation_mse), Float32}, Float32, 4}[Dual{ForwardDiff.Tag{typeof(validation_mse), Float32}}(1.0231731,1.0231731,0.0,0.0,0.0), Dual{ForwardDiff.Tag{typeof(validation_mse), Float32}}(1.0000244,0.0,1.0000244,0.0,0.0), Dual{ForwardDiff.Tag{typeof(validation_mse), Float32}}(0.78652567,0.0,0.0,0.78652567,0.0), Dual{ForwardDiff.Tag{typeof(validation_mse), Float32}}(0.9364305,0.0,0.0,0.0,0.9364305)]
[ Info: 20220524 23:08:47 regression coefficients: ForwardDiff.Dual{ForwardDiff.Tag{typeof(validation_mse), Float32}, Float32, 4}[Dual{ForwardDiff.Tag{typeof(validation_mse), Float32}}(1.0022187,0.002624183,5.3669214e-6,-0.005589499,-0.025572104)]
[ Info: 20220524 23:08:47 training model with parameters ForwardDiff.Dual{ForwardDiff.Tag{typeof(validation_mse), Float32}, ForwardDiff.Dual{ForwardDiff.Tag{typeof(validation_mse), Float32}, Float32, 4}, 4}[Dual{ForwardDiff.Tag{typeof(validation_mse), Float3

...

    19     1.814293e+00     5.850910e-07
 * time: 1299.9149861335754
 * g(x): Float32[1.973485f-8, 2.5867724f-8, -3.2472562f-7, -5.8509096f-7]
 * reached_subproblem_solution: true
 * h(x): Float32[0.01826771 -1.0921008f-8 -0.04582205 -0.20140047; -1.0921013f-8 4.406097f-8 -2.1893514f-7 6.77815f-8; -0.04582205 -2.189353f-7 0.1735949 0.4944557; -0.20140047 6.7781144f-8 0.49445552 2.4567964]
 * x: Float32[-0.29498613, -5.967964, -0.43714696, -0.014979091]
 * lambda: 0.0
 * interior: true
 * hard case: false
 * delta: 1.7884607


[ Info: 20220524 23:29:54 training model with parameters ForwardDiff.Dual{ForwardDiff.Tag{typeof(validation_mse), Float32}, Float32, 4}[Dual{ForwardDiff.Tag{typeof(validation_mse), Float32}}(0.744561,0.744561,0.0,0.0,0.0), Dual{ForwardDiff.Tag{typeof(validation_mse), Float32}}(0.0014392852,0.0,0.0014392852,0.0,0.0), Dual{ForwardDiff.Tag{typeof(validation_mse), Float32}}(0.6458784,0.0,0.0,0.6458784,0.0), Dual{ForwardDiff.Tag{typeof(validation_mse), Float32}}(0.9851343,0.0,0.0,0.0,0.9851343)]
[ Info: 20220524 23:30:09 regression coefficients: ForwardDiff.Dual{ForwardDiff.Tag{typeof(validation_mse), Float32}, Float32, 4}[Dual{ForwardDiff.Tag{typeof(validation_mse), Float32}}(1.0012208,0.0020140908,1.4924186e-8,-0.0035908192,-0.019694353)]
[ Info: 20220524 23:30:10 training model with parameters ForwardDiff.Dual{ForwardDiff.Tag{typeof(validation_mse), Float32}, ForwardDiff.Dual{ForwardDiff.Tag{typeof(validation_mse), Float32}, Float32, 4}, 4}[Dual{ForwardDiff.Tag{typeof(validation_mse), Fl

    20     1.814293e+00     1.566365e-07
 * time: 1382.7439241409302
 * g(x): Float32[3.489497f-12, 9.23058f-9, 9.725228f-8, 1.5663655f-7]
 * reached_subproblem_solution: true
 * h(x): Float32[0.018267618 -6.2314927f-9 -0.04582158 -0.20139964; -6.231494f-9 1.7257607f-8 -5.3962985f-8 4.0998497f-8; -0.04582158 -5.3963f-8 0.17359123 0.49444947; -0.20139964 4.0998497f-8 0.49444947 2.456784]
 * x: Float32[-0.29496047, -6.5436087, -0.4371441, -0.014977308]
 * lambda: 0.0
 * interior: true
 * hard case: false
 * delta: 1.7884607


[ Info: 20220524 23:31:17 training model with parameters ForwardDiff.Dual{ForwardDiff.Tag{typeof(validation_mse), Float32}, Float32, 4}[Dual{ForwardDiff.Tag{typeof(validation_mse), Float32}}(0.74455047,0.74455047,0.0,0.0,0.0), Dual{ForwardDiff.Tag{typeof(validation_mse), Float32}}(0.00085658807,0.0,0.00085658807,0.0,0.0), Dual{ForwardDiff.Tag{typeof(validation_mse), Float32}}(0.6458768,0.0,0.0,0.6458768,0.0), Dual{ForwardDiff.Tag{typeof(validation_mse), Float32}}(0.9851336,0.0,0.0,0.0,0.9851336)]
[ Info: 20220524 23:31:32 regression coefficients: ForwardDiff.Dual{ForwardDiff.Tag{typeof(validation_mse), Float32}, Float32, 4}[Dual{ForwardDiff.Tag{typeof(validation_mse), Float32}}(1.0012202,0.0020140829,8.9824255e-9,-0.0035905798,-0.019694276)]
[ Info: 20220524 23:31:33 training model with parameters ForwardDiff.Dual{ForwardDiff.Tag{typeof(validation_mse), Float32}, ForwardDiff.Dual{ForwardDiff.Tag{typeof(validation_mse), Float32}, Float32, 4}, 4}[Dual{ForwardDiff.Tag{typeof(validation_ms

    21     1.814293e+00     1.828915e-07
 * time: 1465.510311126709
 * g(x): Float32[8.4637755f-9, 3.3822876f-9, -1.4869445f-7, 1.8289153f-7]
 * reached_subproblem_solution: true
 * h(x): Float32[0.018267673 -3.7404218f-9 -0.045821976 -0.2014; -3.740422f-9 6.7599504f-9 3.444311f-10 2.5551211f-8; -0.04582198 3.446117f-10 0.17359361 0.49445462; -0.20140001 2.5551222f-8 0.4944547 2.4567866]
 * x: Float32[-0.29497463, -7.0625534, -0.43714643, -0.014978052]
 * lambda: 0.0
 * interior: true
 * hard case: false
 * delta: 1.7884607


In [9]:
@info "The optimal λ is $λ, found in " * repr(Optim.f_calls(res)) * " function calls"

[ Info: 20220524 23:32:40 The optimal λ is Float32[0.74455047, 0.00085658807, 0.6458768, 0.9851336], found in 22 function calls


In [15]:
stop_criteria = convergence_stopper(1e-6, max_iters = 16)
u, a = train_model(training, stop_criteria, λ);

[ Info: 20220524 23:39:18 training model with parameters Float32[0.74455047, 0.00085658807, 0.6458768, 0.9851336]


In [16]:
#[ Info: 20220522 21:58:57 The optimal λ is Float32[0.74455047, 0.00085658807, 0.6458768, 0.9851336], found in 22 function calls

In [17]:
validation_mse(Optim.minimizer(res))

[ Info: 20220524 23:39:22 training model with parameters Float32[0.74455047, 0.00085658807, 0.6458768, 0.9851336]
[ Info: 20220524 23:39:26 regression coefficients: Float32[0.99968237]


1.8145334f0

## Inference

In [ ]:
model(users, items) = make_prediction(users, items, u, a)
write_alpha(model, residual_alphas, implicit);

[ Info: 20220524 23:40:06 regression coefficients: Float32[0.99968237]
[ Info: 20220524 23:40:06 validation loss: 1.8145334


In [ ]:
write_params(Dict("u" => u, "a" => a, "λ" => λ));